<a href="https://colab.research.google.com/github/mco-gh/pylearn/blob/master/notebooks/9_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 9 - Sample Project

**Make a copy of this notebook by selecting File->Save a copy in Drive from the menu bar above.**

Things you'll learn in this lesson:
- example code and philosophy from a real Python project

[Previous Lesson](https://pylearn.io/lessons/8-Files/)

## Introducing Quizaic

<img height="200" src="https://mco.dev/img/quizaic.webp">

- <a target="_blank" href="https://quizaic.com">Home page</a>
- <a target="_blank" href="https://medium.com/google-cloud/quizaic-a-generative-ai-case-study-ddbd64617395">The technical details</a>
- <a target="_blank" href="https://mco.fyi/genai">Slide deck</a>
- <a target="_blank" href="https://github.com/mco-gh/quizaic">Source code</a>





## Application Architecture

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*mtbUHY9o_E_Nu04MZN6k5Q.png">

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*l-lx-mVy8Kk-NxnpNlMQAQ.png">

## Get a Resource

```
def get(resource_kind, id):
    log(f"Request to get {resource_kind}", severity="INFO")
    if resource_kind not in resource_fields:
        return "Not found", 404, {}

    if resource_kind == "sessions" and id == "me":
        id = get_hashed_email()

    result = db.fetch(resource_kind, id, resource_fields[resource_kind])
    if result is None:
        return "Not found", 404, {}

    return (
        json.dumps(result),
        200,
        {"Content-Type": "application/json", "ETag": base.etag(result)},
    )
```

## Access Checking
```
def user_logged_in(email):
    return email != None


def user_created_quiz(hashed_email, quiz_id):
    if hashed_email is None:
        return False
    quiz = db.fetch("quizzes", quiz_id, ["creator"])
    if quiz and quiz["creator"] == hashed_email:
        return True
    return False
```

## Generate a Quiz

```
   def gen_quiz(
        self,
        topic=BaseQuizgen.TOPIC,
        num_questions=BaseQuizgen.NUM_QUESTIONS,
        num_answers=BaseQuizgen.NUM_ANSWERS,
        difficulty=BaseQuizgen.DIFFICULTY,
        language=BaseQuizgen.LANGUAGE,
        temperature=BaseQuizgen.TEMPERATURE,
    ):
        # print(f"{topic=}, {num_questions=}, {num_answers=}, {difficulty=}, {language=}")
        file_path = os.path.join(os.path.dirname(__file__), f"../prompt.txt")
        with open(file_path, encoding="utf-8") as fp:
            self.prompt_template = fp.read()

        prompt = self.prompt_template.format(
            topic=topic,
            num_questions=num_questions,
            num_answers=num_answers,
            language=language,
            difficulty=difficulty,
        )
        prediction = self.predict_llm(
            MODEL, prompt, temperature, MAX_OUTPUT_TOKENS, TOP_P, TOP_K
        )
        prediction = prediction.strip()
        prediction = re.sub('.*``` *(json)?', '', prediction)
        prediction = prediction[prediction.find('['):]
        parsed = ""
        level = 0
        for i in prediction:
            if i == "[":
                level += 1
            elif i == "]":
                level -= 1
            parsed += i
            if level <= 0:
                break
        prediction = parsed
        print("prediction=", prediction)
        quiz = json.loads(prediction)
        #print(f"{quiz=}")
        # Make sure the correct answer appears randomly in responses
        for i in quiz:
            random.shuffle(i["responses"])
        return quiz
```

[Previous Lesson](https://pylearn.io/lessons/8-Files/)